## Portfolio Optimization Example

Consider the following optimization problem over the portfolio $w\in \mathbb{R}^n$ with expected return $\alpha^\intercal w$, risk aversion factor $\gamma$, asset covariance matrix $\Sigma$, and transaction (short-selling) cost $\kappa_\mathrm{tc(sh)}$:

\begin{align}
\max_{w}. \quad &\alpha^\intercal w - \gamma w^\intercal \Sigma w - \kappa_\mathrm{tc}^\intercal |w-w^\mathrm{prev}| + \kappa_\mathrm{sh}^\intercal (w)_- \\
\text{s.t.} \quad &\mathbb{1}^\intercal w = 1 \\
&\Vert w \Vert_1 \leq L
\end{align}

The previous portfolio is denoted by $w^\mathrm{prev}$, $(\cdot)_-$ represents the argument's negative part, and $L$ is the maximum amount of positions.

With $\Sigma = F \Sigma^f F^\intercal + D$, we can rewrite the problem as,

\begin{align}
\max_{w, f, t_\mathrm{tc}, t_\mathrm{sh}}. \quad &a^\intercal w - \Vert f \Vert_2^2 - \Vert \tilde{D} w\Vert_2^2 - k_\mathrm{tc}^\intercal t_\mathrm{tc} + k_\mathrm{sh}^\intercal t_\mathrm{sh} \\
\text{s.t.} \quad &f = \tilde{F}^\intercal w \\
&\mathbb{1}^\intercal w = 1 \\
&\Vert w \Vert_1 \leq L \\
&|w-w^\mathrm{prev}| \leq t_\mathrm{tc} \\
&(w)_- \geq t_\mathrm{sh} 
\end{align}

parameterized by:

\begin{align}
a &= \frac{\alpha}{\gamma} \\
\tilde{F} &= F \left( \Sigma^f\right)^{1/2} \\
\tilde{D} &= D^{1/2} \\
k_\mathrm{tc} &= \frac{\kappa_\mathrm{tc}}{\gamma} \\
k_\mathrm{sh} &= \frac{\kappa_\mathrm{sh}}{\gamma} \\
&w^\mathrm{prev} \\
&L
\end{align}

Let's define the corresponding CVXPY problem:

In [ ]:
import cvxpy as cp
import numpy as np

# define dimensions
n, m = 100, 10

# define variables
w = cp.Variable(n, name='w')
f = cp.Variable(m, name='f')
t_tc = cp.Variable(n, name='t_tc')
t_sh = cp.Variable(n, name='t_sh')

# define parameters
a = cp.Parameter(n, name='a')
Ft = cp.Parameter((n, m), name='Ft')
Dt = cp.Parameter((n, n), diag=True, name='Dt')
k_tc = cp.Parameter(n, name='k_tc')
k_sh = cp.Parameter(n, name='k_sh')
w_prev = cp.Parameter(n, name='w_prev')
L = cp.Parameter(nonneg=True, name='L')

# define objective
objective = cp.Maximize(a@w
                        -cp.sum_squares(f)
                        -cp.sum_squares(Dt@w)
                        -k_tc@t_tc
                        +k_sh@t_sh)

# define constraints
constraints = [f == Ft.T@w, np.ones(n)@w == 1, 
               cp.norm(w, 1) <= L, 
               cp.abs(w-w_prev)<=t_tc, 
               t_sh<=cp.minimum(0, w)]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
np.random.seed(0)
gamma = 10
alpha = 1000*(-1+2*np.random.rand(n))
kappa_tc = 10*np.ones(n)
kappa_sh = 20*np.ones(n)

a.value = alpha/gamma
Ft.value = np.random.randn(n, m)
Dt.value = np.diag(np.ones(n))
k_tc.value = kappa_tc/gamma
k_sh.value = kappa_sh/gamma
w_prev.value = np.zeros(n)
L.value = 1.6

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='portfolio_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from portfolio_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('portfolio_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
np.random.seed(0)
prob.param_dict['a'].value = alpha/gamma
prob.param_dict['Ft'].value = np.random.randn(n, m)
prob.param_dict['Dt'].value = np.diag(np.ones(n))
prob.param_dict['k_tc'].value = kappa_tc/gamma
prob.param_dict['k_sh'].value = kappa_sh/gamma
prob.param_dict['w_prev'].value = np.zeros(n)
prob.param_dict['L'].value = 1.6

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)